# STRIDE Analyzer — Notebook de Testes

Este notebook contém testes práticos para validar a API `POST /analyze` do projeto **STRIDE Analyzer**.

Instruções rápidas:
1. Preencha as variáveis de ambiente e déploy o serviço (veja README).
2. Atualize `API_URL` com o endpoint local ou remoto.
3. Substitua `diagram.png` por imagens de arquitetura para teste.

O notebook inclui: checks básicos, exemplos de prompt, validação de JSON de saída e testes com múltiplas imagens.


In [ ]:
import requests
import json
from pathlib import Path

# Ajuste este valor para o endpoint onde sua API está rodando
API_URL = 'http://localhost:8080/analyze'

print('API_URL =', API_URL)


In [ ]:
def call_analyze(image_path: str, api_url: str = API_URL):
    """Faz POST multipart para /analyze e retorna JSON ou raise."""
    p = Path(image_path)
    if not p.exists():
        raise FileNotFoundError(f'Arquivo não encontrado: {image_path}')
    with open(p, 'rb') as f:
        files = {'file': (p.name, f, 'image/png')}
        resp = requests.post(api_url, files=files, timeout=60)
    resp.raise_for_status()
    return resp.json()

# Exemplo de chamada (descomente e altere o caminho da imagem)
# res = call_analyze('diagram.png')
# print(json.dumps(res, indent=2, ensure_ascii=False))


## Exemplo de prompt (para validação offline)

Copie este prompt no seu ambiente de testes do Azure OpenAI para verificar o comportamento de geração antes de enviar imagens ao serviço:


In [ ]:
PROMPT_EXAMPLE = '''
You are a security analyst.
Input fields:
CAPTION: Simple web application with LB, API Gateway, Auth Service, Database.
OCR_TEXT: API Gateway
Auth Service
Database
Load Balancer
DETECTED_OBJECTS: load balancer, server, database
Task: Produce a STRIDE analysis per component. Output strictly valid JSON with 'components' array where each component has: name, evidence, stride (Spoofing, Tampering, Repudiation, InfoDisclosure, DoS, Elevation), recommendations (list).
Rules: - Keep recommendations concise (3-6 words each). - Use evidence from caption/OCR/objects.
'''

print(PROMPT_EXAMPLE)


In [ ]:
def validate_stride_structure(obj: dict):
    assert isinstance(obj, dict), 'Resposta não é JSON-obj'
    comps = obj.get('components')
    assert isinstance(comps, list), 'Campo components ausente ou não é lista'
    required_stride = ['Spoofing','Tampering','Repudiation','InfoDisclosure','DoS','Elevation']
    for c in comps:
        assert 'name' in c, 'component sem name'
        assert 'evidence' in c, 'component sem evidence'
        s = c.get('stride')
        assert isinstance(s, dict), 'stride não é dict'
        for k in required_stride:
            assert k in s, f'stride faltando {k}'
    return True

# Exemplo de uso (substitua por saída real)
# example = call_analyze('diagram.png')['stride_analysis']
# validate_stride_structure(example)
# print('Validação OK')


## Testes em lote

Coloque imagens de diagrama em um diretório `tests_images/` e execute a célula abaixo para enviar todas as imagens ao endpoint e salvar respostas em `./test_outputs/`.


In [ ]:
from pathlib import Path

INPUT_DIR = Path('tests_images')
OUTPUT_DIR = Path('test_outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

images = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() in ['.png', '.jpg', '.jpeg']])
print('Encontradas', len(images), 'imagens em', INPUT_DIR)

for img in images:
    try:
        print('Enviando', img)
        res = call_analyze(str(img))
        out_file = OUTPUT_DIR / (img.stem + '.json')
        out_file.write_text(json.dumps(res, ensure_ascii=False, indent=2))
        print('Resposta salva em', out_file)
    except Exception as e:
        print('Erro para', img, ':', e)
